In [9]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime

In [3]:
# with open(r"C:\Users\HP\Downloads\customer_orders.pkl", 'rb') as file:
#     data = pickle.load(file)

# with open(r"C:\Users\HP\Downloads\vip_customers.txt", 'r') as file:
#     vip_cust = file.read()

In [ ]:
# print(data)
# print(type(data))

In [89]:
class CustomerOrderProcessor:
    CATEGORY_MAP = {
        1: 'Electronics',
        2: 'Apparel',
        3: 'Books',
        4: 'Home Goods'
    }

    def __init__(self, pickle_path, vip_path):
        self.pickle_path = pickle_path
        self.vip_path = vip_path
        self.vip_customers = set()
        self.data = []

    def load_vip_customers(self):
        with open(self.vip_path, 'r') as f:
            self.vip_customers = set(int(line.strip()) for line in f if line.strip().isdigit())

    def parse_price(self, price):
        if isinstance(price, str):
            price = price.replace('$', '').replace(',', '')
        try:
            return float(price)
        except:
            return 0.0

    def load_and_flatten_data(self):
        with open(self.pickle_path, 'rb') as f:
            raw_data = pickle.load(f)

        flattened_data = []
        for customer in raw_data:
            customer_id = customer.get('id')
            if customer_id is None:
                continue

            is_vip = int(customer_id) in self.vip_customers
            customer_name = customer.get('name', '')
            registration_date = pd.to_datetime(customer.get('registration_date'), errors='coerce')

            for order in customer.get('orders', []):
                order_id = order.get('order_id')
                order_date = pd.to_datetime(order.get('order_date'), errors='coerce')
                order_total = self.parse_price(order.get('order_total_value'))

                for item in order.get('items', []):
                    unit_price = self.parse_price(item.get('price'))
                    quantity = parse_quantity(item.get('quantity'))
                    total_price = unit_price * quantity
                    percentage = (total_price / order_total * 100) if order_total else 0.0

                    flat_record = {
                        'customer_id': int(customer_id),
                        'customer_name': str(customer_name),
                        'registration_date': registration_date,
                        'is_vip': is_vip,
                        'order_id': parse_order_id(order_id),
                        'order_date': order_date,
                        'product_id': parse_product_id(item.get('item_id')),
                        'product_name': str(item.get('product_name', '')),
                        'category': self.CATEGORY_MAP.get(item.get('category'), 'Misc'),
                        'unit_price': unit_price,
                        'item_quantity': quantity,
                        'total_item_price': total_price,
                        'total_order_value_percentage': round(percentage, 2)
                    }

                    flattened_data.append(flat_record)

        self.data = pd.DataFrame(flattened_data)

    def process(self):
        self.load_vip_customers()
        self.load_and_flatten_data()
        return self.data

In [163]:
materials = CustomerOrderProcessor(
    pickle_path=r"C:\Users\HP\Downloads\customer_orders.pkl",
    vip_path=r"C:\Users\HP\Downloads\vip_customers.txt"
)

data = materials.process()

In [165]:
data.head(10)

,customer_id,customer_name,registration_date,is_vip,order_id,order_date,product_id,product_name,category,unit_price,item_quantity,total_item_price,total_order_value_percentage
0,1,Customer 1,2022-12-31 04:19:19,True,-1,2025-05-03 08:09:20,1,Item 1 for Order 2,Home Goods,52.70,2,105.40,3.81
1,1,Customer 1,2022-12-31 04:19:19,True,-1,2025-05-03 08:09:20,2,Item 2 for Order 2,Misc,498.45,5,2492.25,90.11
2,1,Customer 1,2022-12-31 04:19:19,True,-1,2025-05-03 08:09:20,3,Item 3 for Order 2,Misc,168.15,1,168.15,6.08
3,1,Customer 1,2022-12-31 04:19:19,True,3,2023-09-06 00:42:50,1,Item 1 for Order 3,Electronics,377.96,1,377.96,0.00
4,2,Customer 2,2022-05-27 00:23:28,False,5,2024-08-28 19:37:56,1,Item 1 for Order 5,Misc,342.68,2,685.36,-1198.90
5,2,Customer 2,2022-05-27 00:23:28,False,5,2024-08-28 19:37:56,2,Item 2 for Order 5,Apparel,134.09,1,134.09,-234.56
6,2,Customer 2,2022-05-27 00:23:28,False,5,2024-08-28 19:37:56,3,Item 3 for Order 5,Electronics,295.97,1,295.97,-517.74
7,2,Customer 2,2022-05-27 00:23:28,False,5,2024-08-28 19:37:56,4,Item 4 for Order 5,Misc,316.01,4,1264.04,-2211.18
8,2,Customer 2,2022-05-27 00:23:28,False,6,2024-08-19 12:44:15,1,Item 1 for Order 6,Home Goods,404.43,4,1617.72,78.78
9,2,Customer 2,2022-05-27 00:23:28,False,6,2024-08-19 12:44:15,2,Item 2 for Order 6,Misc,145.27,3,435.81,21.22


In [167]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   customer_id                   198 non-null    int64         
 1   customer_name                 198 non-null    object        
 2   registration_date             193 non-null    datetime64[ns]
 3   is_vip                        198 non-null    bool          
 4   order_id                      198 non-null    int64         
 5   order_date                    192 non-null    datetime64[ns]
 6   product_id                    198 non-null    int64         
 7   product_name                  198 non-null    object        
 8   category                      198 non-null    object        
 9   unit_price                    198 non-null    float64       
 10  item_quantity                 198 non-null    int64         
 11  total_item_price              19

In [55]:
# with open(r"C:\Users\HP\Downloads\customer_orders.pkl", 'rb') as f:
#     data = pickle.load(f)

# print("Type of data:", type(data))
# print("Length of data:", len(data))

# # Try deeper inspection if not empty
# if isinstance(data, dict):
#     print("Top-level keys:", list(data.keys()))
# elif isinstance(data, list) and len(data) > 0:
#     print("Type of first item:", type(data[0]))
#     print("Keys in first item (if dict):", list(data[0].keys()) if isinstance(data[0], dict) else "Not a dict")
#     print("Sample:", data[0])
# else:
#     print("The pickle file appears to be empty or not in expected format.")

Type of data: <class 'list'>
Length of data: 50
Type of first item: <class 'dict'>
Keys in first item (if dict): ['id', 'name', 'registration_date', 'orders']
Sample: {'id': 1, 'name': 'Customer 1', 'registration_date': '2022-12-31 04:19:19', 'orders': [{'order_id': 84, 'order_date': '2024-02-27 21:24:16', 'shipping_address': '840 Main St, City 8', 'order_total_value': 0, 'items': []}, {'order_id': None, 'order_date': '2025-05-03 08:09:20', 'shipping_address': '533 Main St, City 9', 'order_total_value': 2765.8, 'items': [{'item_id': 1, 'product_name': 'Item 1 for Order 2', 'category': 4, 'price': 52.7, 'quantity': 2}, {'item_id': 2, 'product_name': 'Item 2 for Order 2', 'category': 5, 'price': 498.45, 'quantity': 5}, {'item_id': 3, 'product_name': 'Item 3 for Order 2', 'category': 5, 'price': 168.15, 'quantity': 1}]}, {'order_id': 3, 'order_date': '2023-09-06 00:42:50', 'shipping_address': '588 Main St, City 3', 'order_total_value': 0, 'items': [{'item_id': 1, 'product_name': 'Item 1 f

In [169]:
def parse_quantity(value):
    if isinstance(value, int):
        return value
    if isinstance(value, str):
        value = value.strip().lower()

        # Numeric string
        if value.isdigit():
            return int(value)

        # Written numbers
        word_to_num = {
            'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4,
            'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9,
            'ten': 10, 'eleven': 11, 'twelve': 12,
            'thirteen': 13, 'fourteen': 14, 'fifteen': 15,
            'sixteen': 16, 'seventeen': 17, 'eighteen': 18, 'nineteen': 19,
            'twenty': 20, 'free': 1  # Treat 'free' as 1
        }

        return word_to_num.get(value, 0)

    return 0

In [171]:
def parse_product_id(value):
    if isinstance(value, int):
        return value
    if isinstance(value, str):
        value = value.strip().lower()

        # If it's a digit, convert directly
        if value.isdigit():
            return int(value)

        # Convert from words
        word_to_num = {
            'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4,
            'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9,
            'ten': 10, 'eleven': 11, 'twelve': 12, 'thirteen': 13,
            'fourteen': 14, 'fifteen': 15, 'sixteen': 16,
            'seventeen': 17, 'eighteen': 18, 'nineteen': 19,
            'twenty': 20
        }

        return word_to_num.get(value, -1)

    return -1

In [173]:
import re

def parse_order_id(value):
    if isinstance(value, int):
        return value
    if isinstance(value, str):
        # Try to extract the first number in the string
        match = re.search(r'\d+', value)
        if match:
            return int(match.group())
    return -1

In [183]:
CATEGORY_MAP = {
    1: "Electronics",
    2: "Apparel",
    3: "Books",
    4: "Home Goods"
}

def enforce_schema(df: pd.DataFrame) -> pd.DataFrame:
    # --- 1. Clean / map the category column -------------------------------
    # If the column already contains labels, the .map() call leaves them intact
    df["category"] = (
        df["category"]
          .map(CATEGORY_MAP)          # works if the value is 1,2,3,4…
          .fillna(df["category"])     # …otherwise keep what was there
          .fillna("Misc")             # if still NA, set to "Misc"
    )

    # --- 2. Cast every column to its target dtype -------------------------
    # • use nullable versions for ints and bools so NaNs/NA don’t break anything
    dtype_map = {
        "customer_id":                   "Int64",
        "customer_name":                 "string",
        "registration_date":             "datetime64[ns]",   # already okay
        "is_vip":                        "boolean",
        "order_id":                      "Int64",
        "order_date":                    "datetime64[ns]",   # already okay
        "product_id":                    "Int64",
        "product_name":                  "string",
        "category":                      "string",
        "unit_price":                    "float64",
        "item_quantity":                 "Int64",
        "total_item_price":              "float64",
        "total_order_value_percentage":  "float64",
    }

    return df.astype(dtype_map)

In [185]:
# -------------------------------------------------------------------------
data2 = materials.process()      # your existing call
data2 = enforce_schema(data2)     # cast + tidy
print(data2.dtypes)

customer_id                              Int64
customer_name                   string[python]
registration_date               datetime64[ns]
is_vip                                 boolean
order_id                                 Int64
order_date                      datetime64[ns]
product_id                               Int64
product_name                    string[python]
category                        string[python]
unit_price                             float64
item_quantity                            Int64
total_item_price                       float64
total_order_value_percentage           float64
dtype: object


In [151]:
### NOT possible to convert int64 to int as its pandas df default datatype. 

int_cols = data2.select_dtypes(include=["Int64", "int64"]).columns
float_cols  = data2.select_dtypes(include=["float64"]).columns

data2[int_cols] = data2[int_cols].astype(object)
data2[float_cols] = data2[float_cols].astype(object)
print(data.dtypes.head())  

# (float64 columns already match Python float, nothing to do)
print(data2.dtypes)

customer_id                              int32
customer_name                   string[python]
registration_date               datetime64[ns]
is_vip                                 boolean
order_id                                 int64
order_date                      datetime64[ns]
product_id                               int64
product_name                    string[python]
category                        string[python]
unit_price                             float64
item_quantity                            int64
total_item_price                       float64
total_order_value_percentage           float64
dtype: object


In [155]:
data2 = data2.sort_values(by=['customer_id', 'order_id', 'product_id'])

In [157]:
data2.head()

,customer_id,customer_name,registration_date,is_vip,order_id,order_date,product_id,product_name,category,unit_price,item_quantity,total_item_price,total_order_value_percentage
0,1,Customer 1,2022-12-31 04:19:19,True,-1,2025-05-03 08:09:20,1,Item 1 for Order 2,Home Goods,52.70,2,105.40,3.81
1,1,Customer 1,2022-12-31 04:19:19,True,-1,2025-05-03 08:09:20,2,Item 2 for Order 2,Misc,498.45,5,2492.25,90.11
2,1,Customer 1,2022-12-31 04:19:19,True,-1,2025-05-03 08:09:20,3,Item 3 for Order 2,Misc,168.15,1,168.15,6.08
3,1,Customer 1,2022-12-31 04:19:19,True,3,2023-09-06 00:42:50,1,Item 1 for Order 3,Electronics,377.96,1,377.96,0.00
4,2,Customer 2,2022-05-27 00:23:28,False,5,2024-08-28 19:37:56,1,Item 1 for Order 5,Misc,342.68,2,685.36,-1198.90


In [161]:
data2.to_csv(r"C:\Users\HP\Downloads\customer_orders.csv")